# Setup

## Imports

In [ ]:
import bokeh.io
import bokeh.palettes
import bokeh.models

import matplotlib.pyplot as plt

import tables
import pandas as pd

import numpy as np

import fipy

import sys  
import os
import os.path
import glob
import importlib
import itertools

import plotting.plots

import scipy.interpolate
import scipy.integrate

importlib.reload(plotting.plots)

np.seterr(invalid='ignore', divide='ignore')

bokeh.io.output_notebook()

## Parsing and plotting

In [ ]:
global_full_visualization = False

In [ ]:
class PlotSensitivity(plotting.plots.Stacked):
    def __init__(self, x_label, y_labels, x_axis_type = 'linear', **kwargs):
        self.x_axis_type = x_axis_type
        super().__init__(x_label, y_labels, **kwargs)
        
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = color_trace,
                                                 line_color = color_trace,
                                                 **style,)
        
        return [line, circle]
    def _create_figure(self, key_figure):
        return bokeh.plotting.figure(x_axis_type = self.x_axis_type)

In [ ]:
def get_equilibration_voltages(datafile):
    voltages = []
    for simulation in datafile.iter_nodes(where='/simulations'):
        timestep_final = simulation.timesteps[-1]
        
        if timestep_final['time'] < 11.0:
            continue
        voltage = timestep_final['voltage'][-1]
        #print(voltage)
        voltages += [voltage]
    
    return np.array(voltages)

In [ ]:
def plot_equilibration_voltage(datafile):
    voltages = []
    fig,ax = plt.subplots(layout="constrained")
    fig2,ax2 = plt.subplots(layout="constrained")
    for simulation in datafile.iter_nodes(where='/simulations'):
        data = simulation.timesteps.read()
        voltage = data['voltage']
        
        if data['time'].shape[0] < 10 or data['time'][-1] < 11.0:
            continue
        x = simulation.mesh_centers.read()[0]
        ax.plot(x, voltage[-1,:])
        ax2.plot(data['time'], voltage[:,-1])
        
    return data, x
        #print(voltage)
        #voltages += [voltage]
        #fig.show()

In [ ]:
def plot_timeseries_parameter_sweep(datafile,
                                    variation_key,
                                    variation_label,
                                    variation_name,
                                    sensitivity_label = None,
                                    time_separation=10,
                                    plot_crossection = None,
                                    plot_equilibration = None,
                                    plot_rectification = None,
                                    plot_sensitivity = None,
                                    index_not_skipped = None,
                                    labels = None,
                                    height = None,
                                    width = None,
                                    palette = None,
                                    x_axis_type = 'linear',
                                    step_fixed = None,
                                    align_spatial = 'left',
                                    full_visualization = False):
    
    if palette is None:
        palette = bokeh.palettes.viridis(10)

    if plot_crossection is None:
        plot_crossection = plotting.plots.CrossSection(arrangement = [{'x': 't', 'y':['i','in', 'q']},
                                                                      {'x': 'x', 'y':['v', 'c']},],
                                                       x_labels = {'x': r"$$\textsf{Space } (m)$$",
                                                                   't': r"$$\textsf{Time } (s)$$"}, 
                                                       y_labels = {'v': r"$$\textsf{Voltage } (V)$$",
                                                                   'c': r"$$\textsf{Concentration } (\frac{mol}{m^3})$$",
                                                                   'i': r"$$\textsf{Current } (A)$$",
                                                                   'in':r"$$\textsf{Current Normalized} (A)$$",
                                                                   'q': r"$$\textsf{Charge }(C)$$"},
                                                       height = height,
                                                       width = width,
                                                       palette = palette)
        
    if plot_rectification is None:
        plot_rectification = plotting.plots.Chrono(x_label = {'t': r"$$\textsf{Time } (s)$$"}, 
                                                   y_labels = {'ri': r"$${\Large r}_{ I}$$", 
                                                               'rq': r"$${\Large r}_{ Q}$$",
                                                               'tq': r"$$\frac{Q_r}{Q_{f,end}}$$"},
                                                   height = height,
                                                   width = width,
                                                   palette = palette)
    if plot_sensitivity is None:
        plot_sensitivity = PlotSensitivity(x_label = {'var': variation_name}, 
                                           y_labels={'ri': r"$${\Large r}_{I}$$",
                                                     'rq': r"$${\Large r}_{Q}$$",
                                                     't_sr':  r"$${t}_{s,r}$$",
                                                     'i_f': r"$${I}_{f}$$",
                                                    },
                                           height = height,
                                           width = width,
                                           x_axis_type = x_axis_type
                                          )

    time_offset = 0.3

    n=0
    i=0
    varied_attribute_values = []
    metrics = np.zeros((0,5))
    
    for simulation in datafile.iter_nodes(where='/simulations'):
        
        # Don't plot equilibration simulations at the moment
        if "Equilibration" in simulation._v_name:
            continue
        
        if step_fixed is not None:
            step = step_fixed
        elif simulation.timesteps.nrows > 5000:
            step = int(simulation.timesteps.nrows/5000)
        else:
            step = 1

        data = simulation.timesteps.read(step=step)
        
        
        #Only plot complete simulations
        if (data['time'].shape[0] < 10 or data['time'][-1] < 20.0) and not (index_not_skipped is not None and n in index_not_skipped):
            print(f"{simulation._v_name} skipped because not complete")
            continue
        elif index_not_skipped is not None and n not in index_not_skipped:
            n+=1
            print(n)
            continue

        mesh = simulation.mesh_centers.read()
        
        # Align the spatial data to the specified point
        if align_spatial == 'left':
            x_coord_shifted = mesh[0]
        elif align_spatial == 'center':
            x_coord_shifted = mesh[0] - mesh[0,-1]/2 
        
        x_coord_repeated = [x_coord_shifted]*data.shape[0]
        voltage = data['voltage']
        
        time = data['time']
        
        current = data['current']
        charge = scipy.integrate.cumulative_trapezoid(current, time, initial = 0)

        index_start = np.searchsorted(time, time_separation*2 - time_offset, side='right')
        index_mid = np.searchsorted(time, time_separation*3 - time_offset, side='right')
        index_end = np.searchsorted(time, time_separation*4 - time_offset, side='right')

        # Cut out the forward and reverse bias steps
        time_forward = time[index_start : index_mid] - time[index_start]
        time_reverse = time[index_mid : index_end]   - time[index_mid]

        charge_forward = charge[index_start : index_mid] - charge[index_start]
        charge_reverse = charge[index_mid : index_end] - charge[index_mid]
        current_forward = current[index_start : index_mid]
        current_reverse = current[index_mid : index_end]

        # Interpolate the forward bias steps to match the reverse bias
        charge_forward_interp = np.interp(time_reverse, time_forward, charge_forward)
        current_forward_interp = np.interp(time_reverse, time_forward, current_forward)

        rectification_ratio_charge = np.abs(charge_forward_interp/charge_reverse)
        rectification_ratio_current = np.abs(current_forward_interp/current_reverse)

        charge_reversal = -charge_reverse/charge_forward[-1]
        
        concentration_0 = data['concentrations'][:,0,:]
        
        dataframe = bokeh.models.ColumnDataSource({'t': time,
                                                   'x': x_coord_repeated,
                                                   'v': voltage.tolist(),
                                                   'c': concentration_0.tolist(),
                                                   'i': current,
                                                   'in': current/np.abs(current_forward[-1]),
                                                   'q': charge,
                                                  })

        dataframe_rectification = bokeh.models.ColumnDataSource({'t': time_reverse,
                                                                 'rq': rectification_ratio_charge,
                                                                 'ri': rectification_ratio_current,
                                                                 'tq': charge_reversal})
        if variation_key is not None:
            varied_attribute_value = getattr(simulation, variation_key).read()
        else:
            varied_attribute_value = 0
        varied_attribute_values += [varied_attribute_value]

        title = simulation._g_gettitle()
        
        if labels is None and variation_key is not None:
            label = f"{variation_label} = {varied_attribute_value:0.2e}"  
        elif labels is None:
            label = str(n)
        else:
            label = labels[i]
        
        #print(f"{title} with {label} and step = {step}")

        plot_crossection.add_trace(column_data_source = dataframe, label = label, style = {'line_width': 2})
        #plot_iq.add_trace(data=dataframe)
        plot_rectification.add_trace(column_data_source = dataframe_rectification, label = label,  style = {'line_width': 2})
    
        n+=1
        i+=1
        #break
        
        metrics = np.concatenate((metrics, np.array(compute_metrics(data, time_separation))[np.newaxis, :]))
        
        
    dataframe_sensitivity = bokeh.models.ColumnDataSource({'var': varied_attribute_values,
                                                           't_sr': metrics[:, 1],
                                                           'i_f': metrics[:, 2],
                                                           'rq': metrics[:, 3],
                                                           'ri': metrics[:, 4],
                                                          })
    
    style = {'legend_label':sensitivity_label} if sensitivity_label is not None else {}
    style['line_width'] = 2
    
    plot_sensitivity.add_trace(column_data_source = dataframe_sensitivity, style = style)
    
    global global_full_visualization
    
    if full_visualization or global_full_visualization:
        return (plot_crossection, plot_rectification, plot_sensitivity)
    else:
        return (None, None, plot_sensitivity)

In [ ]:
def format_figure(figure, svg = False):
    
    font_size = '15px'
    
    #format font sizes
    figure.yaxis.major_label_text_font_size = font_size
    figure.xaxis.major_label_text_font_size = font_size
    
    figure.xaxis.axis_label_text_font_size = font_size
    figure.yaxis.axis_label_text_font_size = font_size    
    #format tickmarks
    figure.yaxis.minor_tick_in = 4
    figure.yaxis.minor_tick_out = 0
    figure.yaxis.major_tick_in = 8
    figure.yaxis.major_tick_out = 2
    
    figure.xaxis.minor_tick_in = 4
    figure.xaxis.minor_tick_out = 0
    figure.xaxis.major_tick_in = 8
    figure.xaxis.major_tick_out = 2
    
    if svg:
        figure.output_backend = 'svg'
    
def format_plot(plot, svg = False):
    for figure_key in plot.figures:
        format_figure(plot.figures[figure_key], svg)

In [ ]:
def plot_simulation_spatial_data(simulation, plot = None, align_spatial = 'left', label = None, height = 300):

    if plot is None:
        plot = plotting.plots.StackedSlider(x_label = {'x': r"$$\textsf{Space } (m)$$"}, 
                                            y_labels={'v': r"$$\textsf{Voltage } (V)$$",
                                                      'c': r"$$\textsf{Concentration } (\frac{mol}{m^3})$$"},
                                            height = height)
    if label is None:
        label =  simulation._g_gettitle()
        
    data = simulation.timesteps.read()

    mesh = simulation.mesh_centers.read()

    # Align the spatial data to the specified point
    if align_spatial == 'left':
        x_coord_shifted = mesh[0]
    elif align_spatial == 'center':
        x_coord_shifted = mesh[0] - mesh[0,-1]/2 

    x_coord_repeated = [x_coord_shifted]*data['time'].shape[0]
    
    time = data['time']
    #current = data['current']
    voltage = data['voltage']
    
    concentration_0 = data['concentrations'][:,0,:]
    
    dataframe = bokeh.models.ColumnDataSource({'t': time,
                              'x': x_coord_repeated,
                              'v': voltage.tolist(),
                              'c': concentration_0.tolist()})

    plot.add_trace(column_data_source=dataframe, style={'legend_label': label})
    
    return plot

In [ ]:
def plot_simulation_cross_section(simulation,
                                  plot = None,
                                  label = None,
                                  height = 600,
                                  slider_type = 'index',
                                  align_spatial = 'left',
                                  step = None,):
    if plot is None:
        plot = plotting.plots.CrossSection(arrangement = [{'x': 't', 'y':['i', 'jv']},
                                                          {'x': 'x', 'y':['v', 'c']},],
                                           x_labels = {'x': r"$$\textsf{Space } (m)$$",
                                                       't': r"$$\textsf{Time } (s)$$"}, 
                                            y_labels = {'v': r"$$\textsf{Voltage } (V)$$",
                                                        'c': r"$$\textsf{Concentration } (\frac{mol}{m^3})$$",
                                                        'i': r"$$\textsf{Current } (A)$$",
                                                        'jv': r"$$\textsf{Junction Voltage }(V)$$"},
                                            height = height,
                                            slider_type = slider_type)

    if label is None:
        label =  simulation._g_gettitle()

        
    if step is None:
        step = 1
        
    data = simulation.timesteps.read(step = step)
    
    mesh = simulation.mesh_centers.read()
    
    # Align the spatial data to the specified point
    if align_spatial == 'left':
        x_coord_shifted = mesh[0]
    elif align_spatial == 'center':
        x_coord_shifted = mesh[0] - mesh[0,-1]/2 

    x_coord_repeated = [x_coord_shifted]*data['time'].shape[0]
    
    time = data['time']
    current = data['current']
    voltage = data['voltage']
    
    # Bad hacky calculation for where to take voltages. FIX!
    mesh_index_count = mesh.shape[1]
    
    index_0 = int(1/3*mesh_index_count)
    index_1 = int(2/3*mesh_index_count)
    
    junction_voltage =  voltage[:,index_0] - voltage[:,index_1]
    
    concentration_0 = data['concentrations'][:,0,:]
    
    dataframe = bokeh.models.ColumnDataSource({'t': time,
                                               'x': x_coord_repeated,
                                               'v': voltage.tolist(),
                                               'c': concentration_0.tolist(),
                                               'i': current,
                                               'jv': junction_voltage,
                                              })

    plot.add_trace(column_data_source=dataframe, style={'legend_label': label})
    
    return plot

## Metrics

In [ ]:
def get_parameter_sweep_metrics(datafile, variation_key, time_offset = 0.5, time_separation = 10, step_fixed = None):
    
    varied_attribute_values = []
    metrics = np.zeros((0,5))
    
    for simulation in datafile.iter_nodes(where='/simulations'):
        
        # Don't extract equilibration simulations at the moment
        if "Equilibration" in simulation._v_name:
            continue
        
        if step_fixed is not None:
            step = step_fixed
        elif simulation.timesteps.nrows > 5000:
            step = int(simulation.timesteps.nrows/5000)
        else:
            step = 1

        data = simulation.timesteps.read(step=step)
        
        
        #Only extract complete simulations
        if data['time'].shape[0] < 10 or data['time'][-1] < 20.0:
            print(f"{simulation._v_name} skipped because not complete")
            continue
            
        varied_attribute_value = getattr(simulation, variation_key).read()
        
        varied_attribute_values += [varied_attribute_value]
            
        metrics = np.concatenate((metrics, np.array(compute_metrics(data, time_separation))[np.newaxis, :]))
        
        
    data = {'var': np.array(varied_attribute_values),
           't_sr': metrics[:, 1],
           'i_f': metrics[:, 2],
           'rq': metrics[:, 3],
           'ri': metrics[:, 4],
          }
    
    return data

    

In [ ]:
def compute_metrics(data, time_separation, i_f = None, i_r = None):
    
    time_offset = time_separation*0.05

    time = data['time']
    current = data['current']
    charge = scipy.integrate.cumulative_trapezoid(current, time, initial = 0)

    index_start = np.searchsorted(time, time_separation*2, side='right')
    index_mid1 = np.searchsorted(time, time_separation*3 - time_offset, side='right')
    index_mid2 = np.searchsorted(time, time_separation*3, side='right')
    index_end = np.searchsorted(time, time_separation*4 - time_offset, side='right')
    
    # Cut out the forward and reverse bias steps
    time_forward = time[index_start : index_mid1] - time[index_start]
    time_reverse = time[index_mid2 : index_end]   - time[index_mid2]

    charge_forward = charge[index_start : index_mid1] - charge[index_start]
    charge_reverse = charge[index_mid2 : index_end] - charge[index_mid2]
    current_forward = current[index_start : index_mid1]
    current_reverse = current[index_mid2 : index_end]

    # Interpolate the forward bias steps to match the reverse bias
    charge_forward_interp = np.interp(time_reverse, time_forward, charge_forward)
    current_forward_interp = np.interp(time_reverse, time_forward, current_forward)

    
    # Find the settling time:
    settling_accuracy = 0.05

        # Find datapoints that leave the bound
    sign = np.sign(current_forward)
    out_of_bounds_forward = ((sign*current_forward > sign*current_forward[-1]*(1 + settling_accuracy))
                                              | (sign*current_forward < sign*current_forward[-1]*(1 - settling_accuracy)))
                                             
    
    sign = np.sign(current_reverse)
    out_of_bounds_reverse = ((sign*current_reverse > sign*current_reverse[-1]*(1 + settling_accuracy))
                                              | (sign*current_reverse < sign*current_reverse[-1]*(1 - settling_accuracy))
                                             )
    
        # Compute the initial timestep to use as a minimum for the settling time
    t_step_forward = time_forward[1] - time_forward[0]
    t_step_reverse = time_reverse[1] - time_reverse[0]
    
        # Take the last out of bounds index for the settling time
    index_settling_time_forward = np.max(np.nonzero(out_of_bounds_forward), initial = t_step_forward)
    index_settling_time_reverse = np.max(np.nonzero(out_of_bounds_reverse), initial = t_step_reverse)

        # Extract the settling time
    settling_time_forward = time[index_settling_time_forward + index_start] - time_separation*2
    settling_time_reverse = time[index_settling_time_reverse + index_mid2] - time_separation*3

    # Compute the excess reverse bias charge using the settling time indices
    charge_equilibrium = settling_time_reverse*current_reverse[-1]
    
    excess_charge = np.abs((charge_reverse[index_settling_time_reverse] - charge_equilibrium)
                           /charge_equilibrium
                          )
    #print(f"Qr {charge_reverse[index_settling_time_reverse]}, Qe {charge_equilibrium}")
    
    rectification_ratio_current = np.abs(current_forward[-1]/current_reverse[-1])
    
    metrics =  (settling_time_forward,
                settling_time_reverse,
                current_forward[-1],
                excess_charge,
                rectification_ratio_current,)
    
    return metrics
    
    

# Figures

In [ ]:
class PlotSIMetrics(plotting.plots.Columns):
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      line_width = 2,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = color_trace,
                                                 line_color = color_trace,
                                                 size = 4,
                                                 **style,)
        
        return [line, circle]
    
class PlotSIMetricsLinked(plotting.plots.Columns):
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      line_width = 2,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = 'color',
                                                 line_color = color_trace,
                                                 size = 10,
                                                 **style,)
        
        return [line, circle]

## Paper Figures

### Diode Diagram

In [ ]:
data_file_baseline = tables.open_file('./Python output databases/Concentration Boundary/Baseline.hdf5', mode="r")

In [ ]:
simulation = data_file_baseline.list_nodes('/simulations')[0]
data = simulation.timesteps.read()
mesh = simulation.mesh_centers.read()
x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um
time = data['time']
concentrations = data['concentrations']
voltages = data['voltage']
current = data['current']

In [ ]:
plot_paper_2 = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['v', 'c']},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {'v': None,
                                                                     'c': None,
                                                                    },
                                                         height = 350,
                                                         width = 350,
                                                     )

palette = bokeh.palettes.Category10[3]

plot_paper_2.figures['x','v'].line(x = x_coord, y = voltages[-1], line_width = 2, line_color = 'black')

plot_paper_2.figures['x','c'].line(x = x_coord, y = concentrations[-1, 0, :], legend_label = 'Anion', line_width = 2.1, line_color = palette[0])
plot_paper_2.figures['x','c'].line(x = x_coord, y = concentrations[-1, 1, :], legend_label = 'Cation', line_width = 2, line_color = palette[1])

format_plot(plot_paper_2, svg=True)

bokeh.io.show(plot_paper_2.layout)


### Forward vs Reverse

In [ ]:
simulation_internal_path = '/simulations/2023-05-25 16:45:48.299420'

In [ ]:
data_file_baseline = tables.open_file('./Python output databases/Concentration Boundary/Baseline.hdf5', mode="r")

simulation = data_file_baseline.list_nodes('/simulations')[1]
data = simulation.timesteps.read()
mesh = simulation.mesh_centers.read()
x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um
time = data['time']
concentrations = data['concentrations']
voltages = data['voltage']
current = data['current']

In [ ]:

times_forward_stage_a = [19, 20.1, 20.3, 20.6, 21.]
times_forward_stage_b = [21.5, 22.3, 23.5, 25.5, 29]

times_plot_forward = times_forward_stage_a + times_forward_stage_b

palette_forward = bokeh.palettes.Reds[len(times_forward_stage_a)] + bokeh.palettes.Greys[len(times_forward_stage_b)+1][::-1][1:]
#palette_forward = (palette_reverse[-1],) + palette_forward[1:]

times_reverse_stage_a = [29, 30.1, 30.3, 30.6, 31.]
times_reverse_stage_b = [31.2, 32, 33.2, 35, 39]

times_plot_reverse = times_reverse_stage_a + times_reverse_stage_b

palette_reverse= bokeh.palettes.Greys[len(times_reverse_stage_a)+1][0:-1] + bokeh.palettes.Blues[len(times_reverse_stage_b)][::-1]

times_plot = times_plot_reverse + times_plot_forward
palette_combined = palette_reverse + palette_forward
    

plot_paper_3_forward_spatial = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['v', 'c']},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {'v': None,
                                                                     'c': None,
                                                                    },
                                                         height = 550,
                                                         width = 350,
                                                         palette = palette_forward,
                                                     )

plot_paper_3_reverse_spatial = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['v', 'c']},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {'v': None,
                                                                     'c': None,
                                                                    },
                                                         palette = palette_reverse,
                                                         height = 550,
                                                         width = 307,
                                                     )

plot_paper_3_timeseries = plotting.plots.Columns(arrangement = [{'x': 't', 'y':['i']},],
                                                         x_labels = {'t': None,
                                                                    }, 
                                                         y_labels = {'i': None,
                                                                    },
                                                         palette = ['#000000'],
                                                         height = 550,
                                                         width = 350,
                                                     )

plot_legend = bokeh.plotting.figure(width = 200, height = 400) 

currents_plot_forward = []
currents_plot_reverse = []


for plot_time in times_plot_forward:
    index_plot_time = np.searchsorted(time, plot_time, side='right')
    plot_paper_3_forward_spatial.add_trace(x=x_coord,
                                           c=concentrations[index_plot_time, 0, :],
                                           v=voltages[index_plot_time, :],
                                           style = {
                                                    'line_width': 2,}
                                          )
    currents_plot_forward.append(current[index_plot_time])

                                                    
for plot_time in times_plot_reverse:
    index_plot_time = np.searchsorted(time, plot_time, side='right')
    plot_paper_3_reverse_spatial.add_trace(x=x_coord,
                                           c=concentrations[index_plot_time, 0, :],
                                           v=voltages[index_plot_time, :],
                                           style = {
                                                    'line_width': 2,}
                                          )
    currents_plot_reverse.append(current[index_plot_time])
    
currents_plot = currents_plot_reverse + currents_plot_forward
    

times_plot_offset = [t-20 for t in times_plot]
    
plot_paper_3_timeseries.add_trace(t= time - 20, i=current, style = {'line_width': 2})

plot_paper_3_timeseries.figures['t','i'].circle(x=times_plot_offset, y=currents_plot, fill_color=palette_combined, line_color = 'black', size=10)

In [ ]:
format_plot(plot_paper_3_forward_spatial, svg=True)
format_plot(plot_paper_3_reverse_spatial, svg=True)
format_plot(plot_paper_3_timeseries, svg=True)

plot_paper_3_reverse_spatial.figures['x','c'].y_range.start = -0.01 
plot_paper_3_reverse_spatial.figures['x','c'].y_range.end = 0.17
plot_paper_3_reverse_spatial.figures['x','c'].yaxis.visible = False
plot_paper_3_reverse_spatial.figures['x','v'].yaxis.visible = False

plot_paper_3_reverse_spatial.figures['x','v'].y_range.start = -0.17
plot_paper_3_reverse_spatial.figures['x','v'].y_range.end = 0.12

plot_paper_3_forward_spatial.figures['x','c'].y_range.start = -0.01 
plot_paper_3_forward_spatial.figures['x','c'].y_range.end = 0.17

plot_paper_3_forward_spatial.figures['x','v'].y_range.start = -0.17
plot_paper_3_forward_spatial.figures['x','v'].y_range.end = 0.12


plot_paper_3_timeseries.figures['t','i'].x_range.start = 17.5 -20
plot_paper_3_timeseries.figures['t','i'].x_range.end = 41 -20

plot_paper_3_timeseries.figures['t','i'].y_range.start = -0.055
plot_paper_3_timeseries.figures['t','i'].y_range.end = 0.037

layout_figure_2 = bokeh.models.Row(plot_paper_3_timeseries.layout,
                                   bokeh.models.Spacer(width = 50),
                                   plot_paper_3_forward_spatial.layout,
                                   plot_paper_3_reverse_spatial.layout,)

#bokeh.io.show(plot_paper_3_reverse_spatial.layout)
#bokeh.io.show(plot_paper_3_forward_spatial.layout)
#bokeh.io.show(plot_paper_3_timeseries.layout)

bokeh.io.show(layout_figure_2)

### Ionomer Width

In [ ]:
datafile_bath_ionomer_width = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Ionomer Width.hdf5', mode="r")
datafile_bath_ionomer_width_medium_D = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Ionomer Width - Medium D.hdf5', mode="r")
datafile_bath_ionomer_width_high_D = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Ionomer Width - Higher D.hdf5', mode="r")

In [ ]:
class PlotFigure4(plotting.plots.Columns):
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      line_width = 2,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = color_trace,
                                                 line_color = color_trace,
                                                 size = 4,
                                                 **style,)
        
        return [line, circle]

In [ ]:
data_low = get_parameter_sweep_metrics(datafile=datafile_bath_ionomer_width,
                                        variation_key='ionomer_width',
                                        step_fixed=1,)

data_med = get_parameter_sweep_metrics(datafile=datafile_bath_ionomer_width_medium_D,
                                        variation_key='ionomer_width',
                                        step_fixed=1,)


data_high = get_parameter_sweep_metrics(datafile=datafile_bath_ionomer_width_high_D,
                                        variation_key='ionomer_width',
                                        step_fixed=1,)

data_low['var'] = data_low['var']/1e-6
data_med['var'] = data_med['var']/1e-6
data_high['var'] = data_high['var']/1e-6

data = [data_low, data_med, data_high]

D = [1e-12, 1e-11, 1e-10]


source_low = bokeh.models.ColumnDataSource(data_low)
source_med = bokeh.models.ColumnDataSource(data_med)
source_high = bokeh.models.ColumnDataSource(data_high)

plot_paper_4 = PlotFigure4(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                           x_labels = {'var': None,
                                      }, 
                           y_labels = {'ri': " ",
                                       'rq': " ",
                                       't_sr': " ",
                                       'i_f': " ",
                                      },
                           height = 400,
                           width = 750,
                          )

plot_paper_4.add_trace(source_low,
                       #style = {'legend_label': 'D = 5e-12'}
                      )
plot_paper_4.add_trace(source_med,
                       #style = {'legend_label': 'D = 5e-11'}
                      )
plot_paper_4.add_trace(source_high,
                       #style = {'legend_label': 'D = 5e-10'}
                      )


metric_unity_interpolated = np.zeros((3,4))
L_unity = np.zeros((3))
for n, datum in enumerate(data):
    keys = list(datum.keys())
    for i in [0,1,2,3]:
        L_unity[n] = np.sqrt(D[n]*10)/1e-6
        metric_unity_interpolated[n,i] = np.interp(L_unity[n], data[n]['var'], data[n][keys[i+1]])
        figure_key = ('var', keys[i+1])

        plot_paper_4.figures[figure_key].star(x = L_unity[n],
                                              y = metric_unity_interpolated[n, i],
                                              color = 'black',
                                              size = 10,
                                              #legend_label = 'D = 1'
                                             )
        # span = bokeh.models.Span(location = L_unity[n],
        #                                     dimension = 'height',
        #                                     line_color = bokeh.palettes.Category10_3[n],
        #                                     line_width = 2,
        #                                     line_dash = 'dashed'
        #                                     #legend_label = 'D = 1'
        #                         )

        # plot_paper_4.figures[figure_key].add_layout(span)
        
plot_paper_4.figures['var','i_f'].y_range.start = -0.4
plot_paper_4.figures['var','i_f'].y_range.end = 3.7

plot_paper_4.figures['var','ri'].y_range.start = 1
#plot_paper_4.figures['var','ri'].y_range.end = 3.7
        
format_plot(plot_paper_4, svg=True)

bokeh.io.show(plot_paper_4.layout)

### Bath Concentration

In [ ]:
datafile_bath_bath_concentration = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Bath Concentration.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_bath_bath_concentration,
                                        variation_key='bath_concentration',
                                      time_separation= 10)

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {'ri': " ",
                                         'rq': " ",
                                         't_sr': " ",
                                         'i_f': " ",
                                        },
                             height = 400,
                             width = 750,
                            )

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

for key in plot_metrics.figures:
    plot_metrics.figures[key].y_range.start = 0

format_plot(plot_metrics, svg=True)
    
bokeh.io.show(plot_metrics.layout)



### Voltage

In [ ]:
datafile_baths_voltage_longer_time = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Voltage - Even Longer Time.hdf5', mode="r")

In [ ]:
class PlotFigure5(plotting.plots.Columns):
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      line_width = 2,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = 'color',
                                                 line_color = color_trace,
                                                 size = 10,
                                                 **style,)
        
        return [line, circle]
    def _create_figure(self, key_figure):
        return bokeh.plotting.figure()

In [ ]:
metrics = get_parameter_sweep_metrics(datafile=datafile_baths_voltage_longer_time,
                                        variation_key='voltage_peak_peak',
                                      time_separation= 80)

plot_spatial = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['c']},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {
                                                                     'c': None,
                                                                    },
                                                         palette = bokeh.palettes.viridis(10),
                                                         height = 400,
                                                         width = 350,
                                                     )

plot_metrics = PlotFigure5(arrangement = [{'x': 'var', 'y':['ri', 't_sr']},],
                                                         x_labels = {'var': None,
                                                                    }, 
                                                         y_labels = {
                                                                     'ri': None,
                                                                     't_sr': None
                                                                    },
                                                         palette = ('black',),
                                                         height = 400,
                                                         width = 350,
                                                     )



plot_time = 238
voltages_peak_peak = []
for simulation in datafile_baths_voltage_longer_time.iter_nodes(where='/simulations'):
        
        # Don't extract equilibration simulations at the moment
        if "Equilibration" in simulation._v_name:
            continue
        
        if simulation.timesteps.nrows > 5000:
            step = int(simulation.timesteps.nrows/5000)
        else:
            step = 1

        data = simulation.timesteps.read(step=step)
        mesh = simulation.mesh_centers.read()
        x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um
        
        #Only extract complete simulations
        if data['time'].shape[0] < 10 or data['time'][-1] < 20.0:
            print(f"{simulation._v_name} skipped because not complete")
            continue
        
        time = data['time']
        index_plot_time = np.searchsorted(time, plot_time, side='right')
        
        voltage_peak_peak = simulation.voltage_peak_peak.read() 
        voltages_peak_peak.append(voltage_peak_peak)
        
        plot_spatial.add_trace(x = x_coord,
                               c = data['concentrations'][index_plot_time, 0, :],
                               style = {
                                        #'legend_label': f"V = {voltage_peak_peak:.3f}",
                                        'line_width': 2,
                                       }
                              )

metrics['color'] = bokeh.palettes.viridis(10)

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

format_plot(plot_spatial, svg=True)
format_plot(plot_metrics, svg=True)

plot_spatial.figures['x','c'].x_range.start = -4.5
plot_spatial.figures['x','c'].x_range.end = 4.5


plot_paper_5 = bokeh.models.Row(plot_spatial.layout, bokeh.models.Spacer(width = 25), plot_metrics.layout)

bokeh.io.show(plot_paper_5)

### Literature Comparison

In [ ]:
data_file_gilad = tables.open_file('./Python output databases/Literature Comparison/Gilad 27 Forward.hdf5', mode="r")

In [ ]:
data_literature_gilad_main = pd.read_csv('./Experimental Data/gilad manual/Gilad_Diode_Data.csv')
data_literature_gilad_SI = pd.read_csv('./Experimental Data/gilad manual/S5.csv')
simulation = data_file_gilad.list_nodes('/simulations')[-1]

data = simulation.timesteps.read()
mesh = simulation.mesh_centers.read()
x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um
time = data['time']
concentrations = data['concentrations']
voltages = data['voltage']
current = data['current']

index_start = np.searchsorted(time, 500, side = 'right')

data_literature_gilad_main['t'] = data_literature_gilad_main['t'] - 400

A = 25e-6*315e-6
data_literature_gilad_main['I'] = data_literature_gilad_main['I']*1e-6/A
data_literature_gilad_main['I'] = data_literature_gilad_main['I']

data_literature_gilad_SI['if'] = data_literature_gilad_SI['if']*1e-6/A
data_literature_gilad_SI['ir'] = data_literature_gilad_SI['ir']*1e-6/A

data_literature_gilad_SI['tf'] = (data_literature_gilad_SI['tf'] + 500)


plot_gilad_timeseries_SI = plotting.plots.Columns(arrangement = [{'x': 't', 'y':['i']},],
                                                         x_labels = {'t': None,
                                                                    }, 
                                                         y_labels = {'i': None,
                                                                    },
                                                         #palette = ['#000000'],
                                                         height = 550,
                                                         width = 350,
                                                     )


plot_gilad_timeseries_SI.add_trace(t = time[index_start:] - 500,
                                   i = current[index_start:]*1000,
                                   style = {'line_width': 2,
                                            'legend_label': 'Simulation'},
                                  )

datasource_experimental = bokeh.models.ColumnDataSource(data_literature_gilad_SI)

plot_gilad_timeseries_SI.add_trace(datasource_experimental,
                                   t = 'tr',
                                   i = 'ir',
                                   label = "SI",
                                   style = {'line_width': 3,
                                            'line_dash': (3,4),
                                            'legend_label': 'Experiment Reverse'},
                                  )

plot_gilad_timeseries_SI.add_trace(datasource_experimental,
                                   t = 'tf',
                                   i = 'if',
                                   label = "SI",
                                   style = {'line_width': 3,
                                            'line_dash': (3,4),
                                            'legend_label': 'Experiment Forward'},
                                  )

#plot_gilad_timeseries_SI.figures['t','i'].x_range.start = -40
#plot_gilad_timeseries_SI.figures['t','i'].x_range.end = 1000
plot_gilad_timeseries_SI.figures['t','i'].y_range.start = -310
plot_gilad_timeseries_SI.figures['t', 'i'].y_range.end = 400

plot_gilad_timeseries_SI.figures['t', 'i'].legend.location = 'bottom_right'

format_plot(plot_gilad_timeseries_SI, svg=True)

bokeh.io.show(plot_gilad_timeseries_SI.layout)



### Capacitive Electrodes

In [ ]:
datafile_bath_ionomer_width_medium_D = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Ionomer Width - Medium D.hdf5', mode="r")
datafile_microporous_ionomer_width_medium_D = tables.open_file('./Python output databases/Microporous Boundary/Variation - Microporous - Ionomer Width - Medium D.hdf5', mode="r")

In [ ]:
class PlotFigure7(plotting.plots.Columns):
    def _plot_trace(self, data, key_data, key_figure, style, color_trace):
        line = self.figures[key_figure].line(source = data, 
                                      x = key_data[0], 
                                      y = key_data[1],
                                      line_color = color_trace,
                                      line_width = 2,
                                      **style)
        
        circle = self.figures[key_figure].circle(source = data,
                                                 x = key_data[0], 
                                                 y = key_data[1], 
                                                 fill_color = color_trace,
                                                 line_color = color_trace,
                                                 size = 5,
                                                 **style,)
        
        return [line, circle]
    def _create_figure(self, key_figure):
        return bokeh.plotting.figure()

In [ ]:
metrics_microporous = get_parameter_sweep_metrics(datafile=datafile_microporous_ionomer_width_medium_D,
                                        variation_key='ionomer_width',
                                      time_separation= 10)

metrics_concentration = get_parameter_sweep_metrics(datafile=datafile_bath_ionomer_width_medium_D,
                                        variation_key='ionomer_width',
                                      time_separation= 10)

metrics_microporous['var'] = metrics_microporous['var']/1e-6
metrics_concentration['var'] = metrics_concentration['var']/1e-6




In [ ]:
plot_spatial = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['c']},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {
                                                                     'c': None,
                                                                    },
                                                        # palette = bokeh.palettes.viridis(10),
                                                         height = 400,
                                                         width = 350,
                                                     )

plot_metrics = PlotFigure7(arrangement = [{'x': 'var', 'y':['ri', 'i_f']},],
                                                         x_labels = {'var': None,
                                                                    }, 
                                                         y_labels = {
                                                                     'ri': None,
                                                                     'i_f': None
                                                                    },
                                                         #palette = ('black',),
                                                         height = 400,
                                                         width = 350,
                                                     )

simulation_concentration = datafile_bath_ionomer_width_medium_D.list_nodes('/simulations')[7]
simulation_microporous = datafile_microporous_ionomer_width_medium_D.list_nodes('/simulations')[4]

data_concentration = simulation_concentration.timesteps.read()
data_microporous = simulation_microporous.timesteps.read()

mesh_concentration = simulation_concentration.mesh_centers.read()
mesh_microporous = simulation_microporous.mesh_centers.read()

x_coord_concentration = (mesh_concentration[0] - mesh_concentration[0,-1]/2)/1e-6
x_coord_microporous = (mesh_microporous[0] - mesh_microporous[0,-1]/2)/1e-6

plot_time = 28.0

index_plot_time_concentration = np.searchsorted(data_concentration['time'], plot_time, side='right')
index_plot_time_microporous = np.searchsorted(data_microporous['time'], plot_time, side='right')

plot_spatial.add_trace(x = x_coord_concentration,
                       c = data_concentration['concentrations'][index_plot_time_concentration, 0],
                       style = {'line_width': 2,
                                'legend_label': "Baseline",
                               },
                      )
plot_spatial.add_trace(x = x_coord_microporous,
                       c = data_microporous['concentrations'][index_plot_time_microporous, 0],
                       style = {'line_width': 2,
                                'legend_label': "Capacitive",
                               }
                      )

metrics_microporous_data_source = bokeh.models.ColumnDataSource(metrics_microporous)
metrics_concentration_data_source = bokeh.models.ColumnDataSource(metrics_concentration)

plot_metrics.add_trace(metrics_concentration_data_source, 
                       #style = {'legend_label': "Concentration"}
                      )
plot_metrics.add_trace(metrics_microporous_data_source,
                       #style = {'legend_label': "Microporous"}
                      )

format_plot(plot_metrics, svg=True)
format_plot(plot_spatial, svg=True)

plot_spatial.figures['x','c'].x_range.start = -1
plot_spatial.figures['x','c'].x_range.end = 13
#lot_spatial.figures['x','c'].y_range.start = -1
plot_spatial.figures['x','c'].y_range.end = 0.2

plot_paper_7 = bokeh.models.Row(plot_spatial.layout, bokeh.models.Spacer(width = 25), plot_metrics.layout)

bokeh.io.show(plot_paper_7)

## SI Figures

### Voltage Complete

In [ ]:
datafile_baths_voltage_longer_time = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Voltage - Even Longer Time.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_baths_voltage_longer_time,
                                        variation_key='voltage_peak_peak',
                                      time_separation= 80)

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {'ri': " ",
                                         'rq': " ",
                                         't_sr': " ",
                                         'i_f': " ",
                                        },
                             height = 400,
                             width = 750,
                            )

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

format_plot(plot_metrics, svg=True)

bokeh.io.show(plot_metrics.layout)

### Ionomer Concentration

In [ ]:
datafile_bath_ionomer_concentration = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Ionomer Concentration.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_bath_ionomer_concentration,
                                        variation_key='ionomer_concentration',
                                      time_separation= 10)

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {'ri': " ",
                                         'rq': " ",
                                         't_sr': " ",
                                         'i_f': " ",
                                        },
                             height = 400,
                             width = 750,
                            )

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

for key in plot_metrics.figures:
    plot_metrics.figures[key].y_range.start = 0

format_plot(plot_metrics, svg=True)
    
bokeh.io.show(plot_metrics.layout)



### Bath Width

In [ ]:
datafile_baths_bath_width = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Bath Width.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_baths_bath_width,
                                      variation_key='bath_width',
                                      time_separation= 10)

# Convert to microns from meters
metrics['var'] = metrics['var']/1e-6

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {'ri': " ",
                                         'rq': " ",
                                         't_sr': " ",
                                         'i_f': " ",
                                        },
                             height = 400,
                             width = 750,
                            )

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

for key in plot_metrics.figures:
    plot_metrics.figures[key].y_range.start = 0

format_plot(plot_metrics, svg=True)
    
bokeh.io.show(plot_metrics.layout)



### Microporous Ionomer Width

In [ ]:
datafile_microporous_ionomer_width = tables.open_file('./Python output databases/Microporous Boundary/Variation - Microporous - Ionomer Width.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_microporous_ionomer_width,
                                        variation_key='ionomer_width',
                                      time_separation= 10)

# Convert to microns from meters
metrics['var'] = metrics['var']/1e-6

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                          {'x': 'var', 'y':['t_sr','i_f',]}],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {'ri': " ",
                                         'rq': " ",
                                         't_sr': " ",
                                         'i_f': " ",
                                        },
                             height = 400,
                             width = 750,
                            )

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

for key in plot_metrics.figures:
    plot_metrics.figures[key].y_range.start = 0

format_plot(plot_metrics, svg=True)
    
bokeh.io.show(plot_metrics.layout)



### Chemical Reaction speed

In [ ]:
class PlotSIMetricsLinkedLog(PlotSIMetricsLinked):
    def _create_figure(self, key_figure):
        return bokeh.plotting.figure(x_axis_type = 'log')

datafile_agcl_rate_constant = tables.open_file('./Python output databases/Reaction Boundary/Variation - AgCl - Rate Constant - Old.hdf5', mode="r")

metrics = get_parameter_sweep_metrics(datafile=datafile_agcl_rate_constant,
                                        variation_key='rate_constant',
                                      time_separation= 10)

plot_spatial = plotting.plots.Columns(arrangement = [{'x': 'x', 'y':['v', 'c',]},],
                                                         x_labels = {'x': None,
                                                                    }, 
                                                         y_labels = {
                                                                     'c': None,
                                                                     'v': None,
                                                                    },
                                                         palette = bokeh.palettes.viridis(10),
                                                         height = 400,
                                                         width = 350,
                                                     )

plot_metrics = PlotSIMetricsLinkedLog(arrangement = [{'x': 'var', 'y':['ri','rq',]},
                                                     {'x': 'var', 'y':['t_sr','i_f',]}],
                                      x_labels = {'var': None,
                                                 }, 
                                      y_labels = {'ri': " ",
                                                  'rq': " ",
                                                  't_sr': " ",
                                                  'i_f': " ",
                                                 },
                                      palette = ('black',),
                                      height = 400,
                                      width = 750,
                                     )


plot_time = 29.5
rate_constants = []
for simulation in datafile_agcl_rate_constant.iter_nodes(where='/simulations'):
        
        # Don't extract equilibration simulations at the moment
        if "Equilibration" in simulation._v_name:
            continue
        
        if simulation.timesteps.nrows > 5000:
            step = int(simulation.timesteps.nrows/5000)
        else:
            step = 1

        data = simulation.timesteps.read(step=step)
        mesh = simulation.mesh_centers.read()
        x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um
        
        #Only extract complete simulations
        if data['time'].shape[0] < 10 or data['time'][-1] < 20.0:
            print(f"{simulation._v_name} skipped because not complete")
            continue
        
        time = data['time']
        index_plot_time = np.searchsorted(time, plot_time, side='right')
        
        rate_constant = simulation.rate_constant.read() 
        rate_constants.append(rate_constant)
        
        plot_spatial.add_trace(x = x_coord,
                               c = data['concentrations'][index_plot_time, 0, :],
                               v = data['voltage'][index_plot_time, :],
                               style = {
                                        #'legend_label': f"k = {rate_constant:.1e}",
                                        'line_width': 2,
                                       }
                              )

metrics['color'] = bokeh.palettes.viridis(10)

metrics_data_source = bokeh.models.ColumnDataSource(metrics)
        
plot_metrics.add_trace(metrics_data_source,)

for key in plot_metrics.figures:
    plot_metrics.figures[key].y_range.start = 0

format_plot(plot_spatial, svg=True)
format_plot(plot_metrics, svg=True)

#plot_spatial.figures['x','c'].x_range.start = -4.5
#plot_spatial.figures['x','c'].x_range.end = 4.5


plot_combined = bokeh.models.Row(plot_spatial.layout, bokeh.models.Spacer(width = 25), plot_metrics.layout)

bokeh.io.show(plot_combined)

### Chemical Reaction Far Comparison

In [ ]:
datafile_agcl_bath_width = tables.open_file('./Python output databases/Reaction Boundary/Variation - AgCl - Bath Width Old.hdf5', mode="r")
datafile_concentration_bath_width = tables.open_file('./Python output databases/Concentration Boundary/Variation - Baths - Bath Width.hdf5', mode="r")

In [ ]:
plots = plot_timeseries_parameter_sweep(datafile=datafile_agcl_bath_width,
                                variation_key='bath_width',
                                variation_label= "Bath Width",
                                variation_name = r"$$L_{bath}\, (m)$$",
                                time_separation=10, 
                                height = 600,
                                x_axis_type = 'log')

simulation = datafile_agcl_bath_width.list_nodes('/simulations')[-2]
simulation_concentration_bath_width = datafile_concentration_bath_width.get_node('/simulations/width = 3.91e-05')

plot_time = 19

plot_timeseries = plotting.plots.Columns(arrangement = [{'x': 't', 'y':['i']},
                                            ],
                                         y_labels = {'i': " ",},
                                         height = 400,
                                         width = 350,
                             )

plot_spatial = plotting.plots.Columns(arrangement = [
                                             {'x': 'x', 'y':['v', 'c',]},
                                            ],
                                      y_labels = {'v': " ",
                                                  'c': " "
                                                 },
                              height = 400,
                              width = 350,
                             )

data = simulation.timesteps.read(step=1)
mesh = simulation.mesh_centers.read()
x_coord = (mesh[0] - mesh[0,-1]/2)/1e-6 # convert to um

time = data['time']
index_plot_time = np.searchsorted(time, plot_time, side='right')

plot_spatial.add_trace(x = x_coord,
               c = data['concentrations'][index_plot_time, 0, :],
               v = data['voltage'][index_plot_time, :],
               style = {
                        #'legend_label': f"V = {voltage_peak_peak:.3f}",
                        'line_width': 2,
                       }
              )

plot_timeseries.add_trace(x = x_coord,
               t = data['time'],
               i = data['current'],
               style = {
                        #'legend_label': f"V = {voltage_peak_peak:.3f}",
                        'line_width': 2,
                       }
              )

layout = bokeh.models.Row(plot_timeseries.layout, plot_spatial.layout)

# plot = plot_simulation_cross_section(simulation = simulation,
#                                         height = 600,
#                                         label = 'AgCl',
#                                         #slider_type='range',
#                                         align_spatial = 'center',
#                                         # step = 1
#                                     )


# plot_simulation_cross_section(simulation = simulation_concentration_bath_width,
#                                 plot = plot,
#                                 label = 'Concentration',
#                                 #slider_type='range',
#                                 align_spatial = 'center',
#                                 step = 1)


bokeh.io.show(layout)

### Chemical Reaction Voltage

In [ ]:
datafile_agcl_voltage = tables.open_file('./Python output databases/Reaction Boundary/Variation - AgCl - Low Voltage Combined.hdf5', mode="r")

In [ ]:
simulations = datafile_agcl_voltage.list_nodes('/simulations')

plot = None

currents = []
currents_reaction = []
vpps = []
for simulation in simulations:

    data = simulation.timesteps.read()
    time = data['time']
    
    index_end = np.searchsorted(time, 9, side = 'right')
    index_start = np.searchsorted(time, 2.5, side = 'right')

    current = np.mean(data['current'][index_start:index_end])
    currents.append(current)

    voltage = data['voltage']
    vpp = simulation.voltage_peak_peak.read()
    vpps.append(vpp)

    mesh_x = simulation.mesh_faces.read()[0]
    mesh_x_center = simulation.mesh_centers.read()[0]
    mesh_dx = np.diff(mesh_x)
    mesh = fipy.meshes.nonUniformGrid1D.NonUniformGrid1D(dx=mesh_dx)

    voltage_fipy = fipy.CellVariable(mesh = mesh, value = voltage[index_end])

    voltage_fipy.faceValue.constrain(0, where= mesh.facesLeft)
    voltage_fipy.faceValue.constrain(vpp/2, where= mesh.facesRight)

    v_grad = voltage_fipy.grad()[0]


    F = 96485.3321
    RT = 293.15*8.3144598

    current_reaction = (-1e-7
                         * np.exp(
                            v_grad[-1]
                            * 1e-10
                            * 1
                            * F
                            / RT
                        ) + 1e-7
                         * np.exp(
                            v_grad[-1]
                            * 1e-10  # TODO: Replace with proper characteristic length for field
                            * -1
                            * F
                            / RT
                        ))*F*-1
    
    currents_reaction.append(current_reaction)

plot_metrics = PlotSIMetrics(arrangement = [{'x': 'var', 'y':['i_f']},
                                          ],
                             x_labels = {'var': None,
                                        }, 
                             y_labels = {
                                         'i_f': " ",
                                        },
                             height = 200,
                             width = 325,
                            )


metrics_data_source = bokeh.models.ColumnDataSource({'var': vpps, 'i_f': currents_reaction})
        
plot_metrics.add_trace(metrics_data_source,)

format_plot(plot_metrics, svg=True)
    
bokeh.io.show(plot_metrics.layout)




### Mesh spacing

In [ ]:
class PlotMesh(plotting.plots.Columns):
    def _format_figures(self): format_plot(self, svg = True)
    def _create_figure(self, key_figure):
        return bokeh.plotting.figure(y_axis_type = 'log')
    def _plot_trace(self, column_data_source, key_data, key_figure, style, color_trace):
        dots = self.figures[key_figure].circle(
            source=column_data_source,
            x=key_data[0],
            y=key_data[1],
            color=color_trace,
            **style,
        )
        return [dots]

In [ ]:
data_file_gilad = tables.open_file('./Python output databases/Literature Comparison/Gilad 44.hdf5', mode="r")

simulation = data_file_gilad.list_nodes('/simulations')[-1]

mesh = simulation.mesh_faces.read()[0]

plot_mesh = PlotMesh(arrangement = [{'x': 'index', 'y':['spacing']}], height = 400, width = 750)

index = np.arange(0, mesh.shape[0])

charge_density_fixed = simulation.charge_density_fixed.read()
concentrations
plot_mesh.add_trace(index = index, spacing = np.gradient(mesh), style = {'line_dash': 'dashed'})

plot_mesh.show()